In [137]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
import difflib


## Loading CSVs

In [138]:
ml_xlsx_file_path = '/home/celo/Desktop/git-repo-slr/SLR-Automated_selection_of_studies/output-v2/kfold_pearson_fs_test/k1400-report-oct02-10h29m.xlsx'
reviewers_xlsx_file_path = '/home/celo/Desktop/final-changes-reviewers-set/reviewers-matched-table-editted.xlsx'

In [139]:
# Real complete files
if os.path.isfile(ml_xlsx_file_path) and os.path.isfile(reviewers_xlsx_file_path):
    ml_df = pd.read_excel(ml_xlsx_file_path, sheet_name='Probabilities', engine='openpyxl')
    desired_columns = ["Titles", "Claes", "Marcos", "Katia"]
    reviewers_df = pd.read_excel(reviewers_xlsx_file_path, sheet_name='Sheet1', engine='openpyxl', usecols=desired_columns)

else:
    print('[ERROR] Inform valid CSVs files to read')
    raise Exception



In [140]:
# OBS: Titles with typos written on Testing sets 
# # Apply this just to make sure they don't go unoticed

# ml_titles_with_typos = {
#     'A COLABORA{\c{C}}{\~A}O UNIVERSIDADE-EMPRESA NO CONTEXTO DA IND{\'U}STRIA 4.0': 'A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO DA INDÚSTRIA 4.0'
# }
ml_df['Titles'] = ml_df['Titles'].str.replace(r'{\\c{C}}', 'Ç')
ml_df['Titles'] = ml_df['Titles'].str.replace(r'{\\~A}', 'Ã')
ml_df['Titles'] = ml_df['Titles'].str.replace(r'{\'U}', 'Ú')


/tmp/ipykernel_35097/2336510741.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  ml_df['Titles'] = ml_df['Titles'].str.replace(r'{\\c{C}}', 'Ç')
/tmp/ipykernel_35097/2336510741.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  ml_df['Titles'] = ml_df['Titles'].str.replace(r'{\\~A}', 'Ã')
/tmp/ipykernel_35097/2336510741.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  ml_df['Titles'] = ml_df['Titles'].str.replace(r'{\'U}', 'Ú')


In [141]:
# ML's CSV summary
ml_df = ml_df.iloc[: ,1:]  # Only execute this once to remove the first unused column
print('[ML] Num of rows:', len(ml_df))
ml_df.head(15)


[ML] Num of rows: 576


,Titles,Was Selected?,SVM_proba
0,Operations Technology and Organizational Struc...,0,0.989927
1,Design science as nested problem solving,0,0.839662
2,A Large-Scale Empirical Study of Practitioners...,0,0.165938
3,Towards an Approach Matching CMD and DSR to Im...,0,0.997371
4,A PROPOSAL FOR USING DESIGN SCIENCE IN EDUCATI...,0,0.199288
5,University-industry collaboration and open sou...,0,0.963180
6,A bibliometric analysis of the Turkish softwar...,0,0.792188
7,An Improved Indoor Positioning Method Based on...,0,0.789097
8,Industry-academia collaboration in software te...,0,1.000000
9,Measuring Software Reliability: A Trend Using ...,0,0.189447


In [142]:
string_to_numeric = {
    'Assessed, citing SLR': 0,
    'Assessed original SLR': 0,
    'Assessed Batch 1': 0,
    'Assessed': 0,
    'PhD thesis': 0,
    'Bachelor thesis': 0,
    'Licentiate thesis': 0,
    'Not English': 0,
    'Original SLR': 0,
    'Book': 0,
    'Book (chapter in own book)': 0,
    'SMS': 0,
    'SLR': 0,
    'Keynote': 0,
    'See line 55': 0,
    'See line 63': 0,
    'See line 66': 0,
    'See line 108': 0,
    'See line 178': 0,
    'See line 205': 0,
    'Literature review': 0,
    'Review article': 0,
    'Panel summary': 0,
    'Included Assessed original SLR': 2,
    'Included Assessed, citing SLR': 2,
    'Included Assessed': 2
}
for column in ['Claes', 'Marcos', 'Katia']:
    reviewers_df[column] = reviewers_df[column].replace(string_to_numeric)
# OBS: Line 7 should be all 0's after this change

# Reviewers' CSV summary
print('[Reviewers] Num of rows:', len(reviewers_df))
reviewers_df.head(10)

[Reviewers] Num of rows: 576


,Titles,Claes,Marcos,Katia
0,Structuring automotive product lines and featu...,0,0,0
1,Using scrum in outsourced government projects:...,0,0,0
2,Interventions for software security: creating ...,0,0,0
3,Sustainability analysis and ease of learning i...,0,0,0
4,Light-touch interventions to improve software ...,0,0,0
5,Academic and industrial software testing confe...,0,0,0
6,Applying data analytics towards optimized issu...,0,0,0
7,Case studies of industry-academia research col...,1,0,2
8,Wide Band Patch Antenna Structures for Cogniti...,0,0,0
9,Interventions for long‐term software security:...,0,0,0


#



#

### Pandas Functions to Combine CSVs (merge, join...)

In [143]:
# Merge ML's and Reviewers' CSVs into one, by matching Titles
merged_dataframe = pd.merge(ml_df, reviewers_df, on='Titles', how='outer')
print(len(merged_dataframe))
merged_dataframe.head(8)

778


,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia
0,Operations Technology and Organizational Struc...,0.0,0.989927,NaN,NaN,NaN
1,Design science as nested problem solving,0.0,0.839662,NaN,NaN,NaN
2,A Large-Scale Empirical Study of Practitioners...,0.0,0.165938,NaN,NaN,NaN
3,Towards an Approach Matching CMD and DSR to Im...,0.0,0.997371,NaN,NaN,NaN
4,A PROPOSAL FOR USING DESIGN SCIENCE IN EDUCATI...,0.0,0.199288,0.0,0.0,0.0
5,University-industry collaboration and open sou...,0.0,0.963180,NaN,NaN,NaN
6,A bibliometric analysis of the Turkish softwar...,0.0,0.792188,0.0,0.0,0.0
7,An Improved Indoor Positioning Method Based on...,0.0,0.789097,NaN,NaN,NaN


In [144]:
# Add new column with formated titles to sort by it
merged_dataframe['Titles_Formated'] = merged_dataframe['Titles'].str.lower()

# Sorts the csv by title so that missmatch titles are next to each other
merged_dataframe_sorted = merged_dataframe.sort_values(by='Titles_Formated', ascending=True)
merged_dataframe_sorted.head(20)

,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated
150,'Let's not reinvent the wheel': A qualitative ...,0.0,0.996934,0.0,0.0,0.0,'let's not reinvent the wheel': a qualitative ...
6,A bibliometric analysis of the Turkish softwar...,0.0,0.792188,0.0,0.0,0.0,a bibliometric analysis of the turkish softwar...
496,A case study of industry--academia communicati...,1.0,0.992837,NaN,NaN,NaN,a case study of industry--academia communicati...
762,A case study of industry–academia communicatio...,NaN,NaN,2.0,0.0,2.0,a case study of industry–academia communicatio...
48,A case study on artefact-based RE improvement ...,0.0,0.171529,0.0,0.0,0.0,a case study on artefact-based re improvement ...
310,A Chatbot for goal-oriented requirements modeling,0.0,0.427578,0.0,0.0,0.0,a chatbot for goal-oriented requirements modeling
372,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.830361,0.0,0.0,0.0,a colaboração universidade-empresa no contexto...
527,A collaborative autoethnographic analysis of i...,0.0,1.000000,0.0,1.0,2.0,a collaborative autoethnographic analysis of i...
318,A Collaborative Method for Identification and ...,0.0,0.115569,0.0,0.0,0.0,a collaborative method for identification and ...
52,A comparative study of software process improv...,0.0,0.992314,0.0,0.0,0.0,a comparative study of software process improv...


#### Separate the rows containing NaN values from the Titles that were equal in both dataframes

In [145]:
nan_rows = merged_dataframe_sorted.isna().any(axis=1)
missmatched_df = merged_dataframe[nan_rows]
missmatched_df = missmatched_df.sort_values(by='Titles_Formated', ascending=True)
missmatched_df.head()

/tmp/ipykernel_35097/3866861985.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  missmatched_df = merged_dataframe[nan_rows]


,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated
496,A case study of industry--academia communicati...,1.0,0.992837,NaN,NaN,NaN,a case study of industry--academia communicati...
762,A case study of industry–academia communicatio...,NaN,NaN,2.0,0.0,2.0,a case study of industry–academia communicatio...
92,A Critical View on PLM/ALM Convergence in Prac...,0.0,0.635422,NaN,NaN,NaN,a critical view on plm/alm convergence in prac...
622,A critical view on PLM/ALM convergence in prac...,NaN,NaN,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...
681,A dissection of the test-driven development pr...,NaN,NaN,0.0,0.0,0.0,a dissection of the test-driven development pr...


In [146]:
# Resets the index so you can loop throught it using from i=0 to N like an array
missmatched_df = missmatched_df.reset_index()
# missmatched_df = missmatched_df.drop(columns='level_0')
missmatched_df.head(20)


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated
0,496,A case study of industry--academia communicati...,1.0,0.992837,NaN,NaN,NaN,a case study of industry--academia communicati...
1,762,A case study of industry–academia communicatio...,NaN,NaN,2.0,0.0,2.0,a case study of industry–academia communicatio...
2,92,A Critical View on PLM/ALM Convergence in Prac...,0.0,0.635422,NaN,NaN,NaN,a critical view on plm/alm convergence in prac...
3,622,A critical view on PLM/ALM convergence in prac...,NaN,NaN,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...
4,681,A dissection of the test-driven development pr...,NaN,NaN,0.0,0.0,0.0,a dissection of the test-driven development pr...
5,192,A Dissection of the Test-Driven Development Pr...,0.0,0.195613,NaN,NaN,NaN,a dissection of the test-driven development pr...
6,423,A framework to improve university--industry co...,1.0,0.999999,NaN,NaN,NaN,a framework to improve university--industry co...
7,764,A framework to improve university–industry col...,NaN,NaN,0.0,2.0,2.0,a framework to improve university–industry col...
8,668,A large-scale empirical study of practitioners...,NaN,NaN,0.0,0.0,0.0,a large-scale empirical study of practitioners...
9,2,A Large-Scale Empirical Study of Practitioners...,0.0,0.165938,NaN,NaN,NaN,a large-scale empirical study of practitioners...


### Compare strings

In [147]:
# pip install thefuzz
from thefuzz import fuzz, process
from copy import deepcopy

In [148]:
def merge_not_NaN_values(df: pd.DataFrame, index):
    # print(df.loc[index])
    if np.isnan(df.loc[index, 'SVM_proba']) and not(np.isnan(df.loc[index, 'Marcos'])):
        # print('Row 1 is Reviewers and Row 2 is ML')
        return True
    elif np.isnan(df.loc[index, 'Marcos']) and not(np.isnan(df.loc[index, 'SVM_proba'])):
        # print('Row 2 is Reviewers and Row 1 is ML')
        return False
    else:
        raise KeyError('ERROR on merge_not_NaN_values')

In [149]:
# matched_df = merged_dataframe.dropna(subset=merged_dataframe.columns[merged_dataframe.isna().any()].tolist())
i = 0
# similarity_treshold = 85  # 176 missing papers...
similarity_treshold = 75  # 176 missing papers...
# similarity_treshold = 50  # Reduced to 110 missing papers...

similar_matches_df = deepcopy(missmatched_df)
similar_matches_df['Similarity'] = np.nan  # Add new column and set all rows to be NaN

fuzz_match_count = 0
while(i < len(similar_matches_df)-1):
    # print(i)
    curr_row = similar_matches_df.loc[i]['Titles_Formated']
    next_row = similar_matches_df.loc[i+1]['Titles_Formated']

    # Attempt to find match by using the fuzz
    similarity = fuzz.partial_ratio(curr_row, next_row) 

    # TODO: Melhorar esse merge de colunas, se tivesse mais um algo de ML (ex: DT) isso daria errado, o ideal seria usar o nome das colunas e não a posição
    if similarity >= similarity_treshold: # Same title
        fuzz_match_count+=1
        if merge_not_NaN_values(similar_matches_df, i):
            # Copy values from curr_row into next_row for the second and third columns
            similar_matches_df.iloc[i+1, 4:7] = similar_matches_df.iloc[i, 4:7]

            # Copy values from next_row into curr_row for the Reviewers columns
            similar_matches_df.iloc[i, 2:4] = similar_matches_df.iloc[i+1, 2:4]

        else:
            # Copy values from next_row into curr_row for the Reviewers columns
            similar_matches_df.iloc[i, 4:7] = similar_matches_df.iloc[i+1, 4:7]

            # Copy values from curr_row into next_row for the second and third columns
            similar_matches_df.iloc[i+1, 2:4] = similar_matches_df.iloc[i, 2:4]


        # Copy index and title from curr_row to next_row
        similar_matches_df.iloc[i, 0:2] = similar_matches_df.iloc[i+1, 0:2]
        similar_matches_df.at[i, 'Similarity'] = similar_matches_df.at[i+1, 'Similarity'] = similarity
        i+=2

    else: # Not the same
        i+=1

print('Fuzz matches:', fuzz_match_count)
similar_matches_df.head()


Fuzz matches: 198


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
0,762,A case study of industry–academia communicatio...,1.0,0.992837,2.0,0.0,2.0,a case study of industry--academia communicati...,98.0
1,762,A case study of industry–academia communicatio...,1.0,0.992837,2.0,0.0,2.0,a case study of industry–academia communicatio...,98.0
2,622,A critical view on PLM/ALM convergence in prac...,0.0,0.635422,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...,100.0
3,622,A critical view on PLM/ALM convergence in prac...,0.0,0.635422,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...,100.0
4,192,A Dissection of the Test-Driven Development Pr...,0.0,0.195613,0.0,0.0,0.0,a dissection of the test-driven development pr...,100.0


In [150]:
similar_matches_df.head(20)

,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
0,762,A case study of industry–academia communicatio...,1.0,0.992837,2.0,0.0,2.0,a case study of industry--academia communicati...,98.0
1,762,A case study of industry–academia communicatio...,1.0,0.992837,2.0,0.0,2.0,a case study of industry–academia communicatio...,98.0
2,622,A critical view on PLM/ALM convergence in prac...,0.0,0.635422,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...,100.0
3,622,A critical view on PLM/ALM convergence in prac...,0.0,0.635422,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...,100.0
4,192,A Dissection of the Test-Driven Development Pr...,0.0,0.195613,0.0,0.0,0.0,a dissection of the test-driven development pr...,100.0
5,192,A Dissection of the Test-Driven Development Pr...,0.0,0.195613,0.0,0.0,0.0,a dissection of the test-driven development pr...,100.0
6,764,A framework to improve university–industry col...,1.0,0.999999,0.0,2.0,2.0,a framework to improve university--industry co...,96.0
7,764,A framework to improve university–industry col...,1.0,0.999999,0.0,2.0,2.0,a framework to improve university–industry col...,96.0
8,2,A Large-Scale Empirical Study of Practitioners...,0.0,0.165938,0.0,0.0,0.0,a large-scale empirical study of practitioners...,100.0
9,2,A Large-Scale Empirical Study of Practitioners...,0.0,0.165938,0.0,0.0,0.0,a large-scale empirical study of practitioners...,100.0


#### Combine new matches with other dataframe

In [151]:
# Get the first matches
original_matched_df = merged_dataframe_sorted.dropna(how='any')
# print(len(original_matched_df))
# original_matched_df.head()


# Get only the new rows that got a match
new_matches_df = similar_matches_df[(~similar_matches_df['Was Selected?'].isna()) & (~similar_matches_df['SVM_proba'].isna()) 
                                         & (~similar_matches_df['Claes'].isna()) & (~similar_matches_df['Marcos'].isna()) 
                                         & (~similar_matches_df['Katia'].isna())] 
# Drop duplicated rows from new matches (get df with unique Titles)
columns_to_check = ['index', 'Titles']
new_matches_df = new_matches_df[~new_matches_df.duplicated(subset=columns_to_check, keep='first')]
# print(len(new_matches_df))
# new_matches_df.head()              


In [152]:
print(len(original_matched_df))
original_matched_df.head()

374


,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated
150,'Let's not reinvent the wheel': A qualitative ...,0.0,0.996934,0.0,0.0,0.0,'let's not reinvent the wheel': a qualitative ...
6,A bibliometric analysis of the Turkish softwar...,0.0,0.792188,0.0,0.0,0.0,a bibliometric analysis of the turkish softwar...
48,A case study on artefact-based RE improvement ...,0.0,0.171529,0.0,0.0,0.0,a case study on artefact-based re improvement ...
310,A Chatbot for goal-oriented requirements modeling,0.0,0.427578,0.0,0.0,0.0,a chatbot for goal-oriented requirements modeling
372,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.830361,0.0,0.0,0.0,a colaboração universidade-empresa no contexto...


In [153]:
print(len(new_matches_df))
new_matches_df.head()

197


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
0,762,A case study of industry–academia communicatio...,1.0,0.992837,2.0,0.0,2.0,a case study of industry--academia communicati...,98.0
2,622,A critical view on PLM/ALM convergence in prac...,0.0,0.635422,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...,100.0
4,192,A Dissection of the Test-Driven Development Pr...,0.0,0.195613,0.0,0.0,0.0,a dissection of the test-driven development pr...,100.0
6,764,A framework to improve university–industry col...,1.0,0.999999,0.0,2.0,2.0,a framework to improve university--industry co...,96.0
8,2,A Large-Scale Empirical Study of Practitioners...,0.0,0.165938,0.0,0.0,0.0,a large-scale empirical study of practitioners...,100.0


#### Create new DF combining all matches (this should be equal to ALL titles used in the testing set for ML)

In [154]:
# Merge dataframe with all results
columns_to_merge = ['Titles', 'Was Selected?', 'SVM_proba', 'Claes', 'Marcos', 'Katia']
merged_df = original_matched_df.merge(new_matches_df[columns_to_merge], on=columns_to_merge, how='outer')


In [155]:
# Remove the last column 'Titles_Formated' from final dataframe
merged_df = merged_df.drop(columns='Titles_Formated')
print(len(merged_df))
merged_df.head()

571


,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia
0,'Let's not reinvent the wheel': A qualitative ...,0.0,0.996934,0.0,0.0,0.0
1,A bibliometric analysis of the Turkish softwar...,0.0,0.792188,0.0,0.0,0.0
2,A case study on artefact-based RE improvement ...,0.0,0.171529,0.0,0.0,0.0
3,A Chatbot for goal-oriented requirements modeling,0.0,0.427578,0.0,0.0,0.0
4,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.830361,0.0,0.0,0.0


#### Verify DF with unmatched ML titles (this shouldn't happen, check for specific titles to know what went wrong)


In [156]:
# Filtered ML (results that still may be missing...)
not_found_ml_titles_df = similar_matches_df[(~similar_matches_df['Was Selected?'].isna()) & (~similar_matches_df['SVM_proba'].isna()) 
                                         & (similar_matches_df['Claes'].isna()) & (similar_matches_df['Marcos'].isna())]  
print(len(not_found_ml_titles_df))
not_found_ml_titles_df.head(20)

5


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
86,158,Case Studies of Industry-Academia Research Col...,0.0,0.999996,NaN,NaN,NaN,case studies of industry-academia research col...,NaN
115,1,Design science as nested problem solving,0.0,0.839662,NaN,NaN,NaN,design science as nested problem solving,NaN
237,554,Model-based generation of test scripts across ...,0.0,0.993739,NaN,NaN,NaN,model-based generation of test scripts across ...,NaN
263,0,Operations Technology and Organizational Struc...,0.0,0.989927,NaN,NaN,NaN,operations technology and organizational struc...,NaN
318,470,Self-Adaptation Driven by SysML and Goal Model...,0.0,0.896583,NaN,NaN,NaN,self-adaptation driven by sysml and goal model...,NaN


### Use difflib to search for each ML unmatched title the closest match on the REV df

In [157]:
# matched_df = merged_dataframe.dropna(subset=merged_dataframe.columns[merged_dataframe.isna().any()].tolist())
i = 0
# similarity_treshold = 85  
# similarity_treshold = 75  
# similarity_treshold = 50   # 567 matches (only missed 21... REMEMBER THAT ML CSV IS BIGGER THAN REVIEWERS'...)
similarity_treshold = 80   # 567 matches (only missed 21... REMEMBER THAT ML CSV IS BIGGER THAN REVIEWERS'...)
difflib_treshold = similarity_treshold/100


# Format both dataframes to lower case so is easier to match
formated_ml_df = deepcopy(not_found_ml_titles_df)
formated_ml_df['_match_index'] = np.nan  # Add new column and set all rows to be NaN
formated_ml_df = formated_ml_df.drop(['Similarity'], axis=1)
formated_ml_df = formated_ml_df.reset_index()

formated_reviewers_df = deepcopy(reviewers_df)
formated_reviewers_df['Titles_Formated'] = formated_reviewers_df['Titles'].str.lower()
formated_reviewers_df['_match_index'] = np.nan  # Add new column and set all rows to be NaN

difflib_match_count = 0
matches = dict()

while(i < len(formated_ml_df)-1):
# while(i < 3):
    curr_ml_title = formated_ml_df.loc[i]['Titles_Formated']

    # Match by difflib
    difflib_match = difflib.get_close_matches(curr_ml_title, formated_reviewers_df['Titles_Formated'], n=1, cutoff=difflib_treshold)
   
    if difflib_match:
            # print('DIFF LIB:\n\t ml_title: {}\n\tmatch: {}'.format(curr_ml_title, difflib_match))
            difflib_match_count+=1

            # # Use boolean indexing to filter rows where 'Titles' column is equal to 'string_value'
            matched_row_index = formated_reviewers_df[formated_reviewers_df['Titles_Formated'] == difflib_match[0]].index[0]

            # Update the column '_match_index' on both dataframes to use to merge them later
            formated_reviewers_df.at[matched_row_index, '_match_index'] =  difflib_match_count
            formated_ml_df.at[i, '_match_index'] =  difflib_match_count


            if difflib_match_count not in matches:
                  matches[difflib_match_count] = { 'ml_title': curr_ml_title, 'reviewer_match': difflib_match[0] }
    i+=1

print('Difflib matches:', difflib_match_count)
# pd.DataFrame.from_dict(matches).head()


Difflib matches: 2


In [158]:
for c,index in enumerate(matches):
    if c < 7:
        print('ORIGINAL:   ', matches[index]['ml_title'])
        print('REMATCHE:   ', matches[index]['reviewer_match'])
        print('\n')


ORIGINAL:    case studies of industry-academia research collaborations for software development with agile
REMATCHE:    case studies of industry-academia research collaborations for software development with agile


ORIGINAL:    model-based generation of test scripts across product variants: an experience report from the railway industry
REMATCHE:    model‐based generation of test scripts across product variants: an experience report from the railway industry




#### Get the sub dataframe of each one to merge the matches

In [159]:
# Reviewers dataframe
new_re_matches = formated_reviewers_df
new_re_matches = new_re_matches.dropna(subset=['_match_index'])
print('New matches count:', len(new_re_matches))
new_re_matches = new_re_matches.reset_index()
new_re_matches.head()

New matches count: 2


,index,Titles,Claes,Marcos,Katia,Titles_Formated,_match_index
0,7,Case studies of industry-academia research col...,1,0,2,case studies of industry-academia research col...,1.0
1,507,Model‐based generation of test scripts across ...,0,0,0,model‐based generation of test scripts across ...,2.0


In [160]:
# ML dataframe
print('Old missmatch count:', len(formated_ml_df))
new_ml_matches = formated_ml_df
new_ml_matches = new_ml_matches.dropna(subset=['_match_index'])
print('New matches count:', len(new_ml_matches))
new_ml_matches = new_ml_matches.iloc[: ,2:]
new_ml_matches = new_ml_matches.reset_index()
new_ml_matches.head()

Old missmatch count: 5
New matches count: 2


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,_match_index
0,0,Case Studies of Industry-Academia Research Col...,0.0,0.999996,NaN,NaN,NaN,case studies of industry-academia research col...,1.0
1,2,Model-based generation of test scripts across ...,0.0,0.993739,NaN,NaN,NaN,model-based generation of test scripts across ...,2.0


##### Merge the new dataframes to get the new matches

In [161]:
# First join the results
new_matches_df = pd.merge(new_ml_matches.drop(['Claes', 'Marcos', 'Katia', 'index'], axis=1), 
                          new_re_matches.drop(['Titles', 'Titles_Formated', 'index'], axis=1), on='_match_index', how='outer')

print(len(new_matches_df))
new_matches_df.head()

2


,Titles,Was Selected?,SVM_proba,Titles_Formated,_match_index,Claes,Marcos,Katia
0,Case Studies of Industry-Academia Research Col...,0.0,0.999996,case studies of industry-academia research col...,1.0,1,0,2
1,Model-based generation of test scripts across ...,0.0,0.993739,model-based generation of test scripts across ...,2.0,0,0,0


##### Append the new matches to the rest of them

In [162]:
# Now append the new matches to the rest
columns_to_merge = ['Titles', 'Was Selected?', 'SVM_proba', 'Claes', 'Marcos', 'Katia']
merged_df = merged_df.merge(new_matches_df[columns_to_merge], on=columns_to_merge, how='outer')

print(len(merged_df))
merged_df.head()



573


,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia
0,'Let's not reinvent the wheel': A qualitative ...,0.0,0.996934,0.0,0.0,0.0
1,A bibliometric analysis of the Turkish softwar...,0.0,0.792188,0.0,0.0,0.0
2,A case study on artefact-based RE improvement ...,0.0,0.171529,0.0,0.0,0.0
3,A Chatbot for goal-oriented requirements modeling,0.0,0.427578,0.0,0.0,0.0
4,A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO...,0.0,0.830361,0.0,0.0,0.0


### Save file 

In [163]:
RESULTS_PATH = '/home/celo/Desktop/final-changes-reviewers-set'
# csv_file_path = '/home/mcostalonga/new-home/thesis/open-repo-papers-zenodo/zenodo-final-result/output3.csv'
csv_file_path = os.path.join(RESULTS_PATH, 'latest-notebook-result-v3.csv')

# Convert the DataFrame to a CSV file
merged_df.to_csv(csv_file_path, index=False)  # Set index=False to omit the index column

print(f'DataFrame has been successfully saved to {csv_file_path}')

DataFrame has been successfully saved to /home/celo/Desktop/final-changes-reviewers-set/latest-notebook-result-v3.csv


#### Verify AGAIN DF with unmatched ML titles (this shouldn't happen, check for specific titles to know what went wrong)


In [136]:
# ML dataframe

# Filtered ML (results that still may be missing...)
not_found_ml_titles_df = formated_ml_df[(formated_ml_df['_match_index'].isna())]  
print(len(formated_ml_df))
print(len(not_found_ml_titles_df))
not_found_ml_titles_df.head(20)

# print('Old missmatch count:', len(formated_ml_df))
# new_ml_matches = formated_ml_df
# new_ml_matches = new_ml_matches.dropna(subset=['_match_index'])
# print('New matches count:', len(new_ml_matches))
# new_ml_matches = new_ml_matches.iloc[: ,2:]
# new_ml_matches = new_ml_matches.reset_index()
# new_ml_matches.head()

4
3


,level_0,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,_match_index
0,115,1,Design science as nested problem solving,0.0,0.839662,NaN,NaN,NaN,design science as nested problem solving,NaN
2,262,0,Operations Technology and Organizational Struc...,0.0,0.989927,NaN,NaN,NaN,operations technology and organizational struc...,NaN
3,317,470,Self-Adaptation Driven by SysML and Goal Model...,0.0,0.896583,NaN,NaN,NaN,self-adaptation driven by sysml and goal model...,NaN


In [106]:
# TODO: NEW CODE Oct 1 Analysis of missing papers

missing_papers_path = '/home/celo/Desktop/final-changes-reviewers-set'
# csv_file_path = '/home/mcostalonga/new-home/thesis/open-repo-papers-zenodo/zenodo-final-result/output3.csv'
csv_file_path = os.path.join(missing_papers_path, '4_missing_papers.csv')

# Convert the DataFrame to a CSV file
not_found_ml_titles_df.to_csv(csv_file_path, index=False)  # Set index=False to omit the index column
print(f'DataFrame has been successfully saved to {csv_file_path}')


DataFrame has been successfully saved to /home/celo/Desktop/final-changes-reviewers-set/4_missing_papers.csv


In [71]:
formated_ml_df.head()

,level_0,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,_match_index
0,50,359,An Action Research for User Requirements Proce...,0.0,0.500000,NaN,NaN,NaN,an action research for user requirements proce...,NaN
1,89,158,Case Studies of Industry-Academia Research Col...,0.0,0.999996,NaN,NaN,NaN,case studies of industry-academia research col...,1.0
2,118,1,Design science as nested problem solving,0.0,0.839662,NaN,NaN,NaN,design science as nested problem solving,NaN
3,239,554,Model-based generation of test scripts across ...,0.0,0.993739,NaN,NaN,NaN,model-based generation of test scripts across ...,2.0
4,265,0,Operations Technology and Organizational Struc...,0.0,0.989927,NaN,NaN,NaN,operations technology and organizational struc...,NaN


In [ ]:
# # tmp_df = pd.merge(formated_ml_df, formated_reviewers_df, on='_match_index', how='inner')
# tmp_df = pd.merge(formated_ml_df, formated_reviewers_df.drop(['Titles', 'Titles_Formated'], axis=1), on='_match_index', how='inner')

# # tmp_df['_match_index'].fillna(placeholder, inplace=True)
# # tmp_df = tmp_df.dropna(subset=['_match_index'])
# tmp_df = tmp_df.sort_values(by='_match_index', ascending=False)


# # Merge ML's and Reviewers' CSVs into one, by matching Titles
# merged_dataframe = pd.merge(ml_pd, reviewers_pd, on='Titles', how='outer')
# print(len(merged_dataframe))
# merged_dataframe.head(8)

#### Verify DF with unmatched Reviewers' titles (this may happen, reviewers had more titles than our dataset)


In [107]:
# Filtered Reviewers (results that still may be missing...)
not_found_reviewers_titles_df = similar_matches_df[(similar_matches_df['Was Selected?'].isna()) & (similar_matches_df['SVM_proba'].isna()) 
                                         & (~similar_matches_df['Claes'].isna()) & (~similar_matches_df['Marcos'].isna())]  
print(len(not_found_reviewers_titles_df))
not_found_reviewers_titles_df.head(20)




4


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
87,582,Case studies of industry-academia research col...,NaN,NaN,0.0,0.0,0.0,case studies of industry-academia research col...,100.0
88,582,Case studies of industry-academia research col...,NaN,NaN,0.0,0.0,0.0,case studies of industry-academia research col...,100.0
244,756,Model‐based generation of test scripts across ...,NaN,NaN,0.0,0.0,0.0,model‐based generation of test scripts across ...,NaN
393,590,Uma Pesquisa-Ação no Contexto de Processos de ...,NaN,NaN,0.0,0.0,0.0,uma pesquisa-ação no contexto de processos de ...,NaN


In [108]:
# After dropping duplicateds...
unique_not_found_reviewers_titles_df = \
    not_found_reviewers_titles_df[~not_found_reviewers_titles_df.duplicated(subset=columns_to_check, keep='first')]

print(len(unique_not_found_reviewers_titles_df))
unique_not_found_reviewers_titles_df.head(20)

3


,index,Titles,Was Selected?,SVM_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
87,582,Case studies of industry-academia research col...,NaN,NaN,0.0,0.0,0.0,case studies of industry-academia research col...,100.0
244,756,Model‐based generation of test scripts across ...,NaN,NaN,0.0,0.0,0.0,model‐based generation of test scripts across ...,NaN
393,590,Uma Pesquisa-Ação no Contexto de Processos de ...,NaN,NaN,0.0,0.0,0.0,uma pesquisa-ação no contexto de processos de ...,NaN


Fazer comparação de strings row a row:
- Comparar i com i++
- Usar thefuzz

--------------------------------------------------------------------------------------------------------------------

Caso de excesso no ML_pd (papers que precisam ser encontrados no Reviewers_pd mas não foram)
- Pensar em fazer nova busca para encontrá-los
- Was Selected? | SVM_proba != NaN
- Reviewers == NaN

Caso de excesso no Reviewers_pd (papers que foram avaliados pelos revisores mas não foram usados no testing_set)
- Descartar
- Was Selected? | SVM_proba == NaN
- Reviewers != NaN

----------------------------------------------------------------------------------------------------------------------

- Fazer novo merge no pandas que tinha dado match nos títulos (matched_df), pode ser concatenação 
- Reordenar pelos títulos 

Ao final, conferir se todos os X papers do ML.csv estão nesse novo pandas (verificar len)

### TODO: Criar caso de CSV mais complexo e testar para ver se script está funcionando...
- Possíveis problemas: Ordenação (ao ordenar estamos assumindo que os missmatchs estarão sempre um do lado do outro, o que pode não ser verdade caso Um paper tenha um caractere diferente no começo, esteja em lower x upper case...)

### Talvez a solução seja percorrer TODAS as linhas do CSV de REVIEWERS para CADA entrada do CSV de ML....

### Uma solução para o Upper x Lower case seria normalizar tudo para upper ou lower (OBS: Aparentemente o sorted coloca A B C antes de a b c...)

### Testar com os dois CSVs de verdades...

### OBS: By analysing the others notebooks, we selected which xlsx files got the best results for each test case and then manually copied the path of the files here